In [37]:
import pandas as pd
import numpy as np

# Charger le dataset original
df = pd.read_csv('rawg-games-dataset.csv', low_memory=False)
print(f"📊 Dataset original : {len(df):,} jeux | {len(df.columns)} colonnes")

📊 Dataset original : 889,793 jeux | 54 colonnes


---
## Étape 1 : Filtrer les jeux pertinents

**Problème** : 93.5% des jeux n'ont aucune évaluation (ce sont des jeux obscurs sans données utiles)

**Solution** : Garder uniquement les jeux avec au moins 1 vote utilisateur

In [38]:
# Filtrer : garder les jeux avec au moins 1 vote
df_filtered = df[df['ratings_count'].notna() & (df['ratings_count'] > 0)].copy()

print(f"✅ Jeux conservés : {len(df_filtered):,} ({len(df_filtered)/len(df)*100:.1f}%)")
print(f"❌ Jeux supprimés : {len(df) - len(df_filtered):,} (sans évaluation)")

✅ Jeux conservés : 57,839 (6.5%)
❌ Jeux supprimés : 831,954 (sans évaluation)


---
## Étape 2 : Supprimer les colonnes inutiles

### Colonnes supprimées et pourquoi :

| Catégorie | Colonnes | Raison |
|-----------|----------|--------|
| **>80% vides** | reddit_*, clip, tba, metacritic_*, additions_count, parents_count, movies_count, twitch_count, youtube_count, creators_count, game_series_count, reviews_text_count, alternative_names | Données trop incomplètes |
| **Non pertinentes** | saturated_color, dominant_color | Couleurs techniques des images |
| **Stats internes RAWG** | suggestions_count, updated, added, screenshots_count | Pas utiles pour notre API |
| **Redondantes** | description (on garde description_raw), name_original (= name) | Doublons |
| **Dérivées** | released_date, year | Peuvent être recalculées depuis 'released' |

In [39]:
# Colonnes à CONSERVER (liste blanche - plus simple)
cols_to_keep = [
    # Identifiants
    'id', 'slug', 'name',
    # Dates & médias
    'released', 'background_image',
    # Évaluations (les plus remplies)
    'ratings_count', 'reviews_count', 'ratings',
    # Catégorisation
    'platforms', 'parent_platforms', 'genres', 'tags',
    # Infos complémentaires
    'developers', 'publishers', 'stores',
    # Contenu
    'description_raw', 'short_screenshots'
]

# Garder uniquement ces colonnes
cols_to_keep = [c for c in cols_to_keep if c in df_filtered.columns]
df_clean = df_filtered[cols_to_keep].copy()

print(f"✅ Colonnes conservées : {len(df_clean.columns)}")
print(f"❌ Colonnes supprimées : {len(df_filtered.columns) - len(df_clean.columns)}")

✅ Colonnes conservées : 17
❌ Colonnes supprimées : 37


---
## Colonnes conservées (17 colonnes)

| Colonne | Description | % rempli |
|---------|-------------|----------|
| `id` | Identifiant unique | 100% |
| `slug` | URL-friendly (ex: "the-witcher-3") | 100% |
| `name` | Nom du jeu | 100% |
| `released` | Date de sortie | 95% |
| `background_image` | URL image principale | 97% |
| `ratings_count` | Nombre de votes | 100% |
| `reviews_count` | Nombre d'avis | 100% |
| `ratings` | Détail des votes par catégorie | 100% |
| `platforms` | Plateformes (PC, PS5, Xbox...) | 99% |
| `parent_platforms` | Catégories de plateformes | 99% |
| `genres` | Genres (Action, RPG, Indie...) | 93% |
| `tags` | Tags descriptifs | 84% |
| `developers` | Studios de développement | 95% |
| `publishers` | Éditeurs | 84% |
| `stores` | Magasins (Steam, Epic...) | 76% |
| `description_raw` | Description complète | 87% |
| `short_screenshots` | URLs des screenshots | 97% |

In [14]:
# Vérifier le taux de remplissage des colonnes conservées
print("📋 COLONNES FINALES :")
print("=" * 50)
for col in df_clean.columns:
    if df_clean[col].dtype == 'object':
        non_null = df_clean[col].notna() & ~df_clean[col].astype(str).str.lower().isin(['nan', 'none', ''])
        pct = non_null.sum() / len(df_clean) * 100
    else:
        pct = df_clean[col].notna().sum() / len(df_clean) * 100
    status = "✅" if pct > 80 else "⚠️" if pct > 50 else "📉"
    print(f"{status} {col:<25} {pct:>5.1f}%")

📋 COLONNES FINALES :
✅ id                        100.0%
✅ slug                      100.0%
✅ name                      100.0%
✅ released                   94.7%
✅ background_image           97.5%
✅ ratings_count             100.0%
✅ reviews_count              99.9%
✅ ratings                    99.9%
✅ platforms                  99.6%
✅ parent_platforms           99.6%
✅ genres                     92.6%
✅ tags                       83.9%
✅ developers                 94.7%
✅ publishers                 84.2%
⚠️ stores                     76.4%
✅ description_raw            87.0%
✅ short_screenshots          97.5%


---
## Étape 3 : Nettoyer les valeurs

### Conversion en format JSON pour PostgreSQL

Les colonnes multi-valeurs (platforms, genres, developers, etc.) sont converties de :
```
"PC|PlayStation 5|Xbox One"  →  ["PC", "PlayStation 5", "Xbox One"]
```

Avantages en PostgreSQL :
- Recherche précise : `WHERE platforms ? 'PC'`
- Contient tous : `WHERE platforms @> '["PC", "Xbox One"]'`
- Extraction facile : `jsonb_array_elements(platforms)`

In [40]:
import json
import re

# Colonnes multi-valeurs simples (séparées par |, valeurs = strings)
simple_multi_cols = ['platforms', 'parent_platforms', 'genres', 'tags', 'developers', 'publishers', 'stores']

def pipe_to_json_array(value):
    """Convertit 'A|B|C' en '["A", "B", "C"]' (format JSON)"""
    if pd.isna(value) or str(value).lower() in ['nan', 'none', '']:
        return None
    items = [item.strip() for item in str(value).split('|') if item.strip()]
    if not items:
        return None
    return json.dumps(items, ensure_ascii=False)

# Convertir les colonnes simples
for col in simple_multi_cols:
    if col in df_clean.columns:
        before_sample = df_clean[col].dropna().iloc[0] if df_clean[col].notna().any() else "N/A"
        df_clean[col] = df_clean[col].apply(pipe_to_json_array)
        after_sample = df_clean[col].dropna().iloc[0] if df_clean[col].notna().any() else "N/A"
        print(f"✅ {col}")
        print(f"   Avant: {str(before_sample)[:60]}...")
        print(f"   Après: {str(after_sample)[:60]}...")
        print()

# Traitement spécial pour 'ratings' 
def fix_ratings_json(value):
    """Convertit le format ratings en JSON valide"""
    if pd.isna(value) or str(value).lower() in ['nan', 'none', '']:
        return None
    try:
        items = str(value).split('|')
        result = []
        for item in items:
            item = item.strip()
            if not item:
                continue
            # Ajouter guillemets aux clés
            fixed = re.sub(r'(\{|,\s*)(\w+):', r'\1"\2":', item)
            # Ajouter guillemets aux valeurs textuelles
            fixed = re.sub(r':\s*([a-zA-Z][a-zA-Z\s\-]+)([,}])', r': "\1"\2', fixed)
            try:
                obj = json.loads(fixed)
                result.append(obj)
            except:
                continue
        if not result:
            return None
        return json.dumps(result, ensure_ascii=False)
    except:
        return None

if 'ratings' in df_clean.columns:
    before_sample = df_clean['ratings'].dropna().iloc[0] if df_clean['ratings'].notna().any() else "N/A"
    df_clean['ratings'] = df_clean['ratings'].apply(fix_ratings_json)
    after_sample = df_clean['ratings'].dropna().iloc[0] if df_clean['ratings'].notna().any() else "N/A"
    print(f"✅ ratings")
    print(f"   Avant: {str(before_sample)[:70]}...")
    print(f"   Après: {str(after_sample)[:70]}...")
    print()

# Traitement spécial pour short_screenshots: extraire juste les URLs
def extract_screenshot_urls(value):
    """Extrait les URLs des screenshots"""
    if pd.isna(value) or str(value).lower() in ['nan', 'none', '']:
        return None
    # Trouver toutes les URLs https
    urls = re.findall(r'https://[^\s,}]+', str(value))
    if not urls:
        return None
    return json.dumps(urls, ensure_ascii=False)

if 'short_screenshots' in df_clean.columns:
    before_sample = df_clean['short_screenshots'].dropna().iloc[0] if df_clean['short_screenshots'].notna().any() else "N/A"
    df_clean['short_screenshots'] = df_clean['short_screenshots'].apply(extract_screenshot_urls)
    after_sample = df_clean['short_screenshots'].dropna().iloc[0] if df_clean['short_screenshots'].notna().any() else "N/A"
    print(f"✅ short_screenshots (URLs uniquement)")
    print(f"   Avant: {str(before_sample)[:70]}...")
    print(f"   Après: {str(after_sample)[:70]}...")

✅ platforms
   Avant: macOS|Linux|PC...
   Après: ["macOS", "Linux", "PC"]...

✅ parent_platforms
   Avant: PC|Apple Macintosh|Linux...
   Après: ["PC", "Apple Macintosh", "Linux"]...

✅ genres
   Avant: Action...
   Après: ["Action"]...

✅ tags
   Avant: Singleplayer|Multiplayer|Atmospheric|Great Soundtrack|First-...
   Après: ["Singleplayer", "Multiplayer", "Atmospheric", "Great Soundt...

✅ developers
   Avant: Valve Software...
   Après: ["Valve Software"]...

✅ publishers
   Avant: Valve...
   Après: ["Valve"]...

✅ stores
   Avant: Steam...
   Après: ["Steam"]...

✅ ratings
   Avant: {count: 550, id: 4, percent: 47.13, title: recommended}|{count: 263, i...
   Après: [{"count": 550, "id": 4, "percent": 47.13, "title": "recommended"}, {"...

✅ short_screenshots (URLs uniquement)
   Avant: {id: -1, image: https://media.rawg.io/media/games/b7b/b7b8381707152afc...
   Après: ["https://media.rawg.io/media/games/b7b/b7b8381707152afc7d91f5d95de70e...


In [41]:
# Remplacer les 'nan' textuels par de vrais NaN
for col in df_clean.columns:
    if df_clean[col].dtype == 'object':
        df_clean[col] = df_clean[col].replace(['nan', 'NaN', 'None', 'none', ''], np.nan)

# Supprimer les doublons par slug
before = len(df_clean)
df_clean = df_clean.drop_duplicates(subset=['slug'], keep='first')
print(f"🔄 Doublons supprimés : {before - len(df_clean)}")

🔄 Doublons supprimés : 0


---
## Résultat final

In [16]:
print("=" * 60)
print("📊 RÉSUMÉ DU NETTOYAGE")
print("=" * 60)
print(f"")
print(f"   AVANT           →    APRÈS")
print(f"   {len(df):,} jeux      →    {len(df_clean):,} jeux")
print(f"   {len(df.columns)} colonnes     →    {len(df_clean.columns)} colonnes")
print(f"")
print(f"📉 Réduction : {(1 - len(df_clean)/len(df))*100:.1f}% des lignes")
print(f"📉 Réduction : {(1 - len(df_clean.columns)/len(df.columns))*100:.1f}% des colonnes")
print("=" * 60)

📊 RÉSUMÉ DU NETTOYAGE

   AVANT           →    APRÈS
   889,793 jeux      →    57,839 jeux
   54 colonnes     →    17 colonnes

📉 Réduction : 93.5% des lignes
📉 Réduction : 68.5% des colonnes


In [17]:
# Aperçu des données
df_clean.head()

,id,slug,name,released,background_image,ratings_count,reviews_count,ratings,platforms,parent_platforms,genres,tags,developers,publishers,stores,description_raw,short_screenshots
0,19103,half-life-2-lost-coast,Half-Life 2: Lost Coast,2005-10-27,https://media.rawg.io/media/games/b7b/b7b83817...,1164.0,1167.0,"{count: 550, id: 4, percent: 47.13, title: rec...",macOS|Linux|PC,PC|Apple Macintosh|Linux,Action,Singleplayer|Multiplayer|Atmospheric|Great Sou...,Valve Software,Valve,Steam,"Essentially a tech demo, “Half-Life 2: Lost Co...","{id: -1, image: https://media.rawg.io/media/ga..."
1,12020,left-4-dead-2,Left 4 Dead 2,2009-11-17,https://media.rawg.io/media/games/d58/d588947d...,3355.0,3380.0,"{count: 1805, id: 4, percent: 53.4, title: rec...",Xbox 360|Linux|PC|macOS,PC|Xbox|Apple Macintosh|Linux,Action|Shooter,Singleplayer|Steam Achievements|Multiplayer|Fu...,Valve Software|Turtle Rock Studios,Electronic Arts|Valve|Akella,Steam|Xbox 360 Store,Cooperative survival continues with a differen...,"{id: -1, image: https://media.rawg.io/media/ga..."
2,13536,portal,Portal,2007-10-09,https://media.rawg.io/media/games/7fa/7fa0b586...,4896.0,4940.0,"{count: 2960, id: 5, percent: 59.92, title: ex...",macOS|PC|Android|PlayStation 3|Xbox 360|Linux|...,PC|PlayStation|Xbox|Android|Apple Macintosh|Li...,Action|Puzzle,Singleplayer|Steam Achievements|Atmospheric|Gr...,Valve Software|NVIDIA Lightspeed Studios,Valve|Buka Entertainment|NVIDIA|CyberFront,Steam|Google Play,Every single time you click your mouse while h...,"{id: -1, image: https://media.rawg.io/media/ga..."
3,3272,rocket-league,Rocket League,2015-07-07,https://media.rawg.io/media/games/8cc/8cce7c0e...,2827.0,2849.0,"{count: 1561, id: 4, percent: 54.79, title: re...",Nintendo Switch|Linux|macOS|Xbox One|PC|PlaySt...,PC|PlayStation|Xbox|Apple Macintosh|Linux|Nint...,Sports|Racing|Indie,Singleplayer|Steam Achievements|Multiplayer|Fu...,Psyonix,Psyonix,Xbox Store|Steam|PlayStation Store|Nintendo St...,Highly competitive soccer game with rocket-car...,"{id: -1, image: https://media.rawg.io/media/ga..."
4,3439,life-is-strange-episode-1-2,Life is Strange,2015-01-29,https://media.rawg.io/media/games/562/56255381...,3717.0,3764.0,"{count: 1653, id: 5, percent: 43.92, title: ex...",iOS|PC|Linux|PlayStation 3|macOS|Xbox 360|Andr...,PC|PlayStation|Xbox|iOS|Android|Apple Macintos...,Adventure,Singleplayer|Full controller support|Atmospher...,DONTNOD Entertainment,Square Enix|Feral Interactive,GOG|PlayStation Store|Steam|Xbox Store|Google ...,Interactive storytelling and plot-heavy games ...,"{id: -1, image: https://media.rawg.io/media/ga..."


In [42]:
# Sauvegarder le dataset nettoyé
output_file = 'rawg-games-cleaned.csv'
df_clean.to_csv(output_file, index=False)
print(f"💾 Fichier sauvegardé : {output_file}")
print(f"📦 {len(df_clean):,} jeux × {len(df_clean.columns)} colonnes")

💾 Fichier sauvegardé : rawg-games-cleaned.csv
📦 57,839 jeux × 17 colonnes


---
## Étape 4 : Nettoyage final pour PostgreSQL

In [43]:
# Recharger le CSV nettoyé et appliquer des corrections pour PostgreSQL
df_final = pd.read_csv('rawg-games-cleaned.csv', low_memory=False)
print(f"📊 Chargé : {len(df_final):,} jeux")

# 1. Supprimer les lignes où slug ou name est manquant (obligatoires en DB)
before = len(df_final)
df_final = df_final[df_final['slug'].notna() & (df_final['slug'] != '') & (df_final['slug'].astype(str) != 'nan')]
df_final = df_final[df_final['name'].notna() & (df_final['name'] != '') & (df_final['name'].astype(str) != 'nan')]
print(f"✂️ Lignes sans slug/name supprimées : {before - len(df_final)}")

# 2. Convertir les colonnes numériques (enlever les .0)
numeric_cols = ['ratings_count', 'reviews_count']
for col in numeric_cols:
    if col in df_final.columns:
        df_final[col] = pd.to_numeric(df_final[col], errors='coerce').fillna(0).astype(int)

# 3. Supprimer les doublons par slug (clé unique en DB)
before = len(df_final)
df_final = df_final.drop_duplicates(subset=['slug'], keep='first')
print(f"🔄 Doublons slug supprimés : {before - len(df_final)}")

# 4. Nettoyer les caractères problématiques pour CSV/SQL
for col in df_final.columns:
    if df_final[col].dtype == 'object':
        # Remplacer les retours à la ligne par des espaces
        df_final[col] = df_final[col].astype(str).str.replace('\n', ' ', regex=False)
        df_final[col] = df_final[col].astype(str).str.replace('\r', ' ', regex=False)
        # Remettre les vrais NaN
        df_final[col] = df_final[col].replace(['nan', 'None', ''], np.nan)

print(f"✅ Dataset final : {len(df_final):,} jeux")

📊 Chargé : 57,839 jeux
✂️ Lignes sans slug/name supprimées : 2
🔄 Doublons slug supprimés : 0
✅ Dataset final : 57,837 jeux


In [44]:
# Sauvegarder la version finale pour PostgreSQL
output_file = 'rawg-games-cleaned.csv'
df_final.to_csv(output_file, index=False)
print(f"💾 Fichier sauvegardé : {output_file}")
print(f"📦 {len(df_final):,} jeux × {len(df_final.columns)} colonnes")
print(f"\n🚀 Prêt pour Docker : docker compose up --build -d")

💾 Fichier sauvegardé : rawg-games-cleaned.csv
📦 57,837 jeux × 17 colonnes

🚀 Prêt pour Docker : docker compose up --build -d
